# SparkSQL

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
from pyspark import SparkContext

* from pyspark.sql import SparkSession: Mengimpor SparkSession, yang merupakan titik awal utama untuk bekerja dengan DataFrame dan API SQL di Spark.
* from pyspark.sql.types import *: Mengimpor semua tipe data dari modul pyspark.sql.types. Ini berguna jika Anda perlu mendefinisikan skema khusus untuk DataFrame.
* from pyspark.sql.functions import *: Mengimpor semua fungsi bawaan yang berguna untuk melakukan operasi pada DataFrame, seperti agregasi, transformasi, dll.
* from datetime import datetime: Mengimpor kelas datetime dari modul Python standar datetime untuk bekerja dengan tanggal dan waktu.
* from pyspark import SparkContext: Mengimpor SparkContext, yang merupakan titik awal untuk Spark. Ini bertanggung jawab untuk mengelola koneksi ke cluster dan membuat RDD.

In [ ]:
sc = SparkContext()

* sc = SparkContext(): Membuat objek SparkContext, yang diperlukan untuk bekerja dengan RDD di Spark.

In [ ]:
spark = SparkSession.builder.appName("SparkSQL-basic").getOrCreate()

* spark = SparkSession.builder.appName("SparkSQL-basic").getOrCreate(): Membuat objek SparkSession dengan nama aplikasi "SparkSQL-basic". SparkSession adalah antarmuka utama untuk bekerja dengan DataFrame dan API SQL di Spark. Jika sesi Spark sudah ada, ia akan menggunakan sesi yang sudah ada, jika tidak, sesi baru akan dibuat.

## Membuat Data (RDD)

Kode berikut merupakan contoh membuat RDD yang berisi dua catatan siswa dengan berbagai atribut. RDD ini kemudian dikonversi menjadi DataFrame, yang memungkinkan Anda untuk menampilkan data dalam format tabel yang rapi. Dua kali panggilan ke show() menunjukkan isi DataFrame, sekali dengan pemotongan nilai yang panjang dan sekali tanpa pemotongan.

In [ ]:
students_records = sc.parallelize([Row(roll_no=1, name='Intan', passed=True, marks={'Math':89,'Phycsics':87,'Chemistry':81},
                                       sports=['chess','foot-ball'], Dob=datetime(2012,5,1,13,1,5)),
                                   Row(roll_no=2, name='Smith', passed=False, marks={'Math':29,'Phycsics':31,'Chemistry':36},
                                       sports=['volley-ball','table-tennis'], Dob=datetime(2012,5,12,14,2,5))])

* sc.parallelize([...]): Mendistribusikan koleksi data (dalam hal ini, daftar Row) ke dalam RDD. Setiap Row mewakili catatan siswa dengan berbagai atribut seperti roll_no, name, passed, marks, sports, dan Dob.
* Row(...): Row adalah cara untuk membuat baris data yang berisi berbagai nilai dengan nama kolom yang terkait. Di sini, kita memiliki dua Row, masing-masing mewakili satu catatan siswa dengan atribut yang ditentukan.

In [ ]:
student_records_df = students_records.toDF()
student_records_df.show()

* students_records.toDF(): Mengonversi RDD yang berisi Row menjadi DataFrame. DataFrame adalah struktur data yang lebih terstruktur, mirip dengan tabel di database relasional, yang mendukung berbagai operasi SQL.
* student_records_df.show(): Menampilkan konten DataFrame dalam bentuk tabel, menampilkan baris pertama dengan jumlah kolom yang terbatas dan nilai yang dipotong jika terlalu panjang.

In [ ]:
student_records_df.show(truncate=False)

* student_records_df.show(truncate=False): Menampilkan konten DataFrame tanpa memotong nilai yang panjang di kolom mana pun, sehingga seluruh nilai dapat dilihat dengan jelas.

## Membuat View

Kode berikut menunjukkan bagaimana membuat tampilan sementara (temporary view) dari DataFrame untuk memungkinkan eksekusi query SQL secara langsung di atasnya. Hasil query dapat langsung ditampilkan menggunakan .show() atau disimpan ke dalam variabel untuk digunakan lebih lanjut. Tipe data yang dihasilkan oleh query SQL dalam Spark adalah DataFrame.

In [ ]:
student_records_df.createOrReplaceTempView('records')

* createOrReplaceTempView('records'): Baris ini membuat atau mengganti bila sudah ada tampilan sementara (temporary view) bernama 'records' dari DataFrame student_records_df. Tampilan sementara ini memungkinkan untuk menjalankan query SQL di atas DataFrame menggunakan sintaks SQL biasa. Tampilan ini hanya tersedia selama sesi Spark saat ini aktif dan akan hilang bila sesinya mati/putus.

In [ ]:
spark.sql("SELECT * FROM records").show()

* spark.sql("SELECT * FROM records").show(): Menggunakan metode spark.sql() untuk menjalankan query SQL "SELECT * FROM records", yang akan memilih semua kolom dari tampilan 'records'. Metode .show() kemudian digunakan untuk menampilkan hasil query tersebut dalam format tabel.

In [ ]:
spark.sql("SELECT name FROM records").show()

* spark.sql("SELECT name FROM records").show(): Query SQL ini hanya memilih kolom name dari tampilan 'records' dan menampilkan hasilnya. Jadi, hasil yang akan ditampilkan adalah nama-nama siswa dalam catatan.

In [ ]:
type(1)

* type(1): Ini adalah perintah Python biasa yang digunakan untuk mengecek tipe data dari nilai 1. Hasilnya akan menunjukkan bahwa tipe data 1 adalah int (integer).

In [ ]:
results = spark.sql("SELECT * FROM records")
type(results)

* results = spark.sql("SELECT * FROM records"): Di sini, hasil dari query SQL "SELECT * FROM records" disimpan ke dalam variabel results. Hasilnya tidak langsung ditampilkan tetapi disimpan untuk digunakan lebih lanjut.
* type(results): Baris ini memeriksa tipe data dari variabel results. Karena results menyimpan hasil dari query SQL yang dilakukan pada DataFrame, tipe data yang dihasilkan akan menjadi pyspark.sql.dataframe.DataFrame. Dengan kata lain, results adalah sebuah DataFrame.

## Filter Data


Baris-baris kode ini menunjukkan beberapa query SQL yang dijalankan di atas DataFrame yang telah diubah menjadi tampilan sementara bernama records.

In [ ]:
spark.sql('SELECT roll_no, marks["Phycsics"], sports[1] FROM records').show()

* roll_no: Nomor rol siswa.
* marks["Phycsics"]: Nilai siswa dalam mata pelajaran Fisika (Phycsics). Di sini, marks adalah kolom bertipe Map (mirip dengan dictionary di Python), dan "Phycsics" adalah kunci yang digunakan untuk mengakses nilai Fisika.
* sports[1]: Elemen kedua dari daftar sports, di mana sports adalah kolom bertipe Array (mirip dengan list di Python). Indeks 1 digunakan karena indeks di SQL dimulai dari 0, sehingga elemen pertama memiliki indeks 0 dan elemen kedua memiliki indeks 1.

In [ ]:
spark.sql('SELECT * FROM records WHERE passed=True').show()

* WHERE passed=True: Query ini memilih semua kolom (*) dari tampilan records tetapi membatasi hanya untuk baris-baris di mana kolom passed bernilai True. Ini berarti hanya siswa yang lulus yang akan ditampilkan dalam hasil.

In [ ]:
spark.sql('SELECT * FROM records WHERE marks["Chemistry"] < 40').show()

* WHERE marks["Chemistry"] < 40: Query ini juga memilih semua kolom (*) dari tampilan records, tetapi hanya untuk baris-baris di mana nilai kimia (Chemistry) kurang dari 40. Di sini, kolom marks adalah Map, dan "Chemistry" adalah kunci yang digunakan untuk memeriksa nilai kimia. Query ini akan mengembalikan siswa yang gagal dalam mata pelajaran kimia.

## Type Data 

Tipe data di Spark membantu mendefinisikan struktur DataFrame, memastikan bahwa data diproses dengan cara yang benar. Dengan memahami tipe data ini, Anda dapat membuat DataFrame yang sesuai dengan kebutuhan analisis data Anda.

### Data dasar

* StringType: teks atau string
* IntegerType: angka bulat
* LongType: angka bulat lebih dari integer
* FloatType: angka desimal presisi tunggal (32-bit) lebih cepat dan menghemat ruang, tetapi hanya dapat menyimpan sekitar 7 digit desimal dengan tingkat presisi yang lebih rendah. Cocok untuk aplikasi di mana presisi tinggi tidak begitu kritis dengan rentang -10e38 hingga 10e38
* DoubleType: angka desimal presisi ganda (64-bit) memiliki presisi yang lebih tinggi, dapat menyimpan hingga 16 digit desimal, dan digunakan dalam aplikasi yang memerlukan akurasi tinggi dengan rentang -10e308 hingga 10e308
* BooleanType: True atau False
* DateType: tanggal
* TimestampType: tanggal dan waktu
* BinaryType: data biner (byte array)
* DecimalType: desimal dengan presisi tetap dan skala  untuk memberikan kontrol penuh atas bagaimana angka desimal disimpan, diproses, dan ditampilkan, terutama dalam aplikasi keuangan dan ilmiah di mana akurasi numerik sangat penting.
* NullType: merepresentasikan nilai null atau tipe tidak diketahui
* ByteType: angka kecil (8-bit signed integer)
* ShortType: angka bulat kecil (16-bit signed integer) rentang -32,768 hingga 32,767

### Data kompleks

* ArrayType: daftar dari elemen dengan satu type tertentu, misal kolom sports yang hanya berisi teks
* MapType: merepresentasikan peta atau dictionary dengan pasangan kunci-nilai, misal kolom marks.
* StructType: merepresentasikan struktur data seperti record atau row yang memiliki beberapa kolom dengan tipe data yang berbeda
* StructField: Digunakan bersama StructType untuk mendefinisikan nama, tipe, dan apakah kolom tersebut nullable (bisa bernilai null) atau tidak


## Global View

In [ ]:
student_records_df.createGlobalTempView('global_records')

* createGlobalTempView('global_records'): membuat sebuah global temporary view dengan nama 'global_records' dari DataFrame student_records_df.
* Global Temporary View:
  * Berbeda dengan temporary view biasa, global temporary view ini dapat diakses dari semua sesi Spark yang aktif, bukan hanya sesi di mana view tersebut dibuat.
  * View ini berada di dalam database khusus bernama global_temp.
  * Global temporary view akan tetap ada selama sesi Spark yang membuatnya masih aktif. Setelah sesi tersebut berakhir, view ini akan hilang.

In [ ]:
spark.sql('SELECT * FROM global_temp.global_records').show()

* spark.sql('SELECT * FROM global_temp.global_records'): Baris ini menjalankan query SQL yang mengambil semua kolom (*) dari global temporary view yang baru saja dibuat, yaitu global_records, yang berada di dalam namespace global_temp.
* global_temp.global_records: Karena ini adalah global temporary view, nama view harus disertai dengan nama database global_temp, sehingga path lengkapnya adalah global_temp.global_records.

## Hapus Kolom

In [ ]:
student_records_df.columns

* menampilkan daftar nama kolom dari DataFrame student_records_df. Hasilnya adalah list yang berisi nama-nama kolom yang ada dalam DataFrame tersebut.

In [ ]:
student_records_df = student_records_df.drop('passed')
student_records_df.show()

* drop('passed'): Baris ini menghapus kolom passed dari DataFrame student_records_df.
* DataFrame yang baru tanpa kolom passed disimpan kembali ke variabel student_records_df.

In [ ]:
student_records_df.createOrReplaceTempView('records_new')
spark.sql('SELECT * FROM records_new').show()

* createOrReplaceTempView('records_new'): Baris ini membuat atau mengganti temporary view bernama records_new dari DataFrame student_records_df yang telah diperbarui.
* Temporary view ini hanya akan tersedia selama sesi Spark aktif.

In [ ]:
spark.sql('SELECT * FROM records').show()

* spark.sql('SELECT * FROM records_new'): Menjalankan query SQL untuk memilih semua kolom dari temporary view yang baru dibuat records_new.

## Agregat = Kolom Baru

* Menghitung Rata-Rata dengan SQL: Menggunakan query SQL untuk menghitung rata-rata nilai dari beberapa kolom dan menampilkan hasilnya.
* Menambahkan Kolom ke DataFrame Baru: Menggunakan query SQL untuk menghitung rata-rata nilai dan menambahkannya sebagai kolom baru dalam DataFrame, kemudian menampilkan DataFrame yang diperbarui.

In [ ]:
student_records_df.show(truncate=False)

* Cek data

In [ ]:
spark.sql('SELECT round((marks.Phycsics+marks.Chemistry+marks.Math)/3) avg_marks FROM records').show()

* spark.sql('SELECT round((marks.Phycsics+marks.Chemistry+marks.Math)/3) avg_marks FROM records'): Menjalankan query SQL untuk menghitung rata-rata nilai (avg_marks) dari mata pelajaran Phycsics, Chemistry, dan Math untuk setiap siswa.
* round(): Fungsi ini membulatkan hasil rata-rata ke bilangan bulat terdekat.

In [ ]:
student_records_df = spark.sql('SELECT *, round((marks.Phycsics+marks.Chemistry+marks.Math)/3) avg_marks FROM records')
student_records_df.show()

* spark.sql('SELECT *, round((marks.Phycsics+marks.Chemistry+marks.Math)/3) avg_marks FROM records'): Menjalankan query SQL untuk memilih semua kolom dari temporary view records dan menambahkan kolom baru avg_marks yang berisi rata-rata nilai dari mata pelajaran Phycsics, Chemistry, dan Math. Nilai rata-rata ini dibulatkan ke bilangan bulat terdekat.
* student_records_df =: Menyimpan hasil query ke dalam DataFrame baru student_records_df, yang sekarang mencakup kolom tambahan avg_marks.

## Update View

In [ ]:
student_records_df.createOrReplaceTempView('records_new')
spark.sql('SELECT * FROM records_new').show()

* createOrReplaceTempView('record_new'): Baris ini membuat atau mengganti temporary view dengan nama 'record_new' dari DataFrame student_records_df.
* Temporary view ini hanya akan tersedia selama sesi Spark aktif. Dengan view ini, Anda dapat menjalankan query SQL terhadap DataFrame akan merujuk ke view yang baru.

In [ ]:
student_records_df = student_records_df.withColumn('status', (when(col('avg_marks')>=40, 'passed')).otherwise('failed'))
student_records_df.show()

* withColumn('status', ...): Baris ini menambahkan kolom baru bernama 'status' ke DataFrame student_records_df.
* when(col('avg_marks') >= 40, 'passed'): Fungsi when digunakan untuk memberikan nilai 'passed' jika nilai di kolom avg_marks lebih besar atau sama dengan 40.
* otherwise('failed'): Jika kondisi when tidak terpenuhi (yaitu avg_marks kurang dari 40), maka nilai 'failed' akan diberikan ke kolom 'status'.
* Hasilnya adalah DataFrame baru yang sekarang mencakup kolom status yang menunjukkan apakah nilai rata-rata setiap siswa memenuhi syarat untuk dianggap lulus atau tidak.

## Membuat Dataframe Baru

In [ ]:
employeeData = (('John','HR','NY',90000,34,10000),('Neha','HR','NY',86000,28,20000),
                ('Robert','Sales','CA',81000,56,22000),('Maria','Sales','CA',99000,45,15000),
                ('Paul','IT','NY',98000,38,14000),('Jen','IT','CA',90000,34,20000),
                ('Raj','IT','CA',93000,28,28000),('Pooja','IT','CA',95000,31,19000))
columns = ('employee_name','department','state','salary','age','bonus')
employeeDf = spark.createDataFrame(employeeData, columns)
employeeDf.show()

* employeeData: Tuple berisi data karyawan, di mana setiap tuple mewakili satu baris data dengan informasi tentang nama karyawan, departemen, negara bagian, gaji, usia, dan bonus.
* columns: Tuple berisi nama-nama kolom yang sesuai dengan data dalam employeeData. Kolom-kolom ini mencakup nama karyawan, departemen, negara bagian, gaji, usia, dan bonus.
* spark.createDataFrame(employeeData, columns): Baris ini membuat DataFrame Spark dari data karyawan (employeeData) dan menetapkan nama kolom (columns).
* DataFrame employeeDf akan memiliki struktur yang ditentukan oleh tuple columns.


## GroupBy

In [ ]:
employeeDf.groupBy(col('department')).agg(sum(col('salary'))).show()

* groupBy(col('department')): Mengelompokkan data berdasarkan kolom department. Artinya, data akan dikelompokkan menurut nilai unik dalam kolom department.
* agg(sum(col('salary'))): Menghitung jumlah (sum) dari kolom salary untuk setiap kelompok departemen.
* Mengelompokkan data per departemen dan menghitung total gaji.

In [ ]:
employeeDf.groupBy(col('department')).agg(sum(col('salary')).alias('total_sal')).orderBy('total_sal').show()

* agg(sum(col('salary')).alias('total_sal')): Menghitung jumlah gaji dan memberi alias 'total_sal' pada kolom hasil agregasi.
* orderBy('total_sal'): Mengurutkan hasil agregasi berdasarkan kolom total_sal (total gaji) dalam urutan menaik.
* Mengelompokkan data per departemen, menghitung total gaji, dan mengurutkan hasilnya.

In [ ]:
employeeDf.groupBy(col('department')).agg(sum(col('salary')).alias('total_sal')).orderBy(col('total_sal').desc()).show()

* orderBy(col('total_sal').desc()): Mengurutkan hasil agregasi berdasarkan kolom total_sal (total gaji) dalam urutan menurun.
* Mengelompokkan data per departemen, menghitung total gaji, dan mengurutkan hasilnya secara menurun.

In [ ]:
employeeDf.groupBy(col('department'),col('state')).agg(sum(col('bonus'))).show()

* groupBy(col('department'), col('state')): Mengelompokkan data berdasarkan kolom department dan state. Artinya, data akan dikelompokkan menurut kombinasi unik dari nilai dalam kedua kolom tersebut.
* agg(sum(col('bonus'))): Menghitung jumlah (sum) dari kolom bonus untuk setiap kelompok yang terdiri dari kombinasi departemen dan negara bagian.
* Mengelompokkan data per kombinasi departemen dan negara bagian, serta menghitung total bonus.

In [ ]:
employeeDf.groupBy(col('department')).agg(avg(col('salary')).alias('avg_salary'),max(col('bonus')).alias('max_bonus')).show()

* agg(avg(col('salary')).alias('avg_salary'), max(col('bonus')).alias('max_bonus')): Menghitung rata-rata (avg) dari kolom salary dan nilai maksimum (max) dari kolom bonus. Memberi alias 'avg_salary' untuk rata-rata gaji dan 'max_bonus' untuk bonus maksimum.
*  Mengelompokkan data per departemen, menghitung rata-rata gaji, dan bonus maksimum.

## Windowing

Kelas Window dalam PySpark digunakan untuk mendefinisikan operasi jendela (window) yang memungkinkan perhitungan agregat dan peringkat pada subset data dalam DataFrame. Ini memungkinkan Anda untuk melakukan operasi yang kompleks dan  melibatkan grup data tanpa perlu menggabungkan atau menyaring data secara terpisah.

In [ ]:
from pyspark.sql.window import Window
windowSpec = Window.partitionBy("department").orderBy(col("salary").desc())
employeeDf = employeeDf.withColumn("rank", dense_rank().over(windowSpec))
employeeDf.filter(col("rank") == 2).show()

In [ ]:
employeeDf.filter(col("rank") == 1).show()

* from pyspark.sql.window import Window: Mengimpor kelas Window dari modul pyspark.sql.window, yang digunakan untuk mendefinisikan jendela (window) untuk operasi agregasi dan peringkat (ranking) dalam DataFrame.
* Window.partitionBy("department"): Mendefinisikan jendela yang membagi data berdasarkan kolom department. Artinya, perhitungan berikutnya akan dilakukan terpisah untuk setiap departemen.
* orderBy(col("salary").desc()): Mengurutkan data dalam setiap partisi (departemen) berdasarkan kolom salary dalam urutan menurun (desc).
* withColumn("rank", dense_rank().over(windowSpec)): Menambahkan kolom baru bernama rank ke DataFrame employeeDf. Kolom ini berisi peringkat berdasarkan jendela yang didefinisikan.
* dense_rank(): Fungsi ini digunakan untuk memberikan peringkat pada baris dalam setiap partisi. dense_rank memberikan peringkat yang berurutan tanpa celah, artinya jika dua baris memiliki peringkat yang sama, peringkat berikutnya akan dilompati.
* filter(col("rank") == 2): Menyaring DataFrame untuk memilih baris di mana kolom rank memiliki nilai 2. Ini menunjukkan karyawan yang memiliki peringkat kedua dalam setiap departemen berdasarkan gaji.

## Join

In [ ]:
managers = (('Sales','Maria'),('HR','John'),('IT','Pooja'))
mg_columns = ('department','manager')
managerDf = spark.createDataFrame(managers, mg_columns)
managerDf.show()

* managers: Tuple berisi data tentang manajer, di mana setiap tuple mewakili satu baris data dengan informasi tentang departemen dan nama manajer.
* mg_columns: Tuple berisi nama-nama kolom yang sesuai dengan data dalam managers. Kolom-kolom ini mencakup department (departemen) dan manager (nama manajer).
* spark.createDataFrame(managers, mg_columns): Baris ini membuat DataFrame Spark dari data manajer (managers) dan menetapkan nama kolom (mg_columns). DataFrame managerDf akan memiliki struktur yang ditentukan oleh tuple mg_columns.

In [ ]:
employeeDf.join(managerDf, employeeDf['department'] == managerDf['department'], how='inner').select(col('employee_name'), col('manager')).show()

In [ ]:
employeeDf.join(managerDf, employeeDf['department'] == managerDf['department'], how='inner').show()

* employeeDf.join(managerDf, employeeDf['department'] == managerDf['department'], how='inner'): Melakukan operasi join antara employeeDf dan managerDf menggunakan kolom department sebagai kunci penggabungan.
  * Kondisi join berdasarkan kolom department di kedua DataFrame.
  * Jenis join yang digunakan adalah inner join. Artinya, hanya baris yang memiliki kecocokan di kedua DataFrame yang akan disertakan dalam hasil akhir.
  * Baris dari employeeDf dan managerDf yang memiliki department yang sama akan digabungkan.
* .select(col('employee_name'), col('manager')): Memilih kolom employee_name dari employeeDf dan kolom manager dari managerDf untuk ditampilkan dalam hasil akhir. 

## Terima kasih

In [ ]:
spark.stop()